In [1]:
# Generate Missing Power BI Datasets
# From your existing 5 files

import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("GENERATING MISSING POWER BI DATASETS")
print("="*70)

# Load existing files
df_monthly = pd.read_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/monthly_financials.csv")
df_detailed = pd.read_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/initiative_monthly_detail.csv")
df_initiatives = pd.read_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/initiative_master.csv")
df_uncertainty = pd.read_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/roi_uncertainty_analysis.csv")

# Convert dates
df_monthly['month'] = pd.to_datetime(df_monthly['month'])
df_detailed['month'] = pd.to_datetime(df_detailed['month'])

print("✅ Loaded existing files")

# ============================================
# DATASET 1: MASTER METRICS
# ============================================

latest_month = df_monthly['month'].max()
latest = df_monthly[df_monthly['month'] == latest_month].iloc[0]

master_metrics = pd.DataFrame([
    {'metric': 'Current Runway', 'value': latest['runway_months'], 'unit': 'months'},
    {'metric': 'Cash Balance', 'value': latest['cash_balance'], 'unit': 'dollars'},
    {'metric': 'Monthly Burn', 'value': latest['net_burn'], 'unit': 'dollars'},
    {'metric': 'Total Initiatives', 'value': len(df_initiatives), 'unit': 'count'},
])

master_metrics.to_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/powerbi_master_metrics.csv", index=False)
print("✅ Created: powerbi_master_metrics.csv")

# ============================================
# DATASET 2: RUNWAY TREND
# ============================================

runway_trend = df_monthly[['month', 'cash_balance', 'runway_months', 
                           'net_burn', 'total_burn', 'total_revenue']].copy()
runway_trend.to_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/powerbi_runway_trend.csv", index=False)
print("✅ Created: powerbi_runway_trend.csv")

# ============================================
# DATASET 3: INITIATIVE SCORECARD
# ============================================

latest_detailed = df_detailed[df_detailed['month'] == latest_month].copy()

latest_detailed['roi_percent'] = np.where(
    latest_detailed['monthly_burn'] > 0,
    ((latest_detailed['monthly_revenue'] - latest_detailed['monthly_burn']) / 
     latest_detailed['monthly_burn']) * 100,
    0
)

# Add recommendation
def get_recommendation(roi):
    if roi < -50:
        return 'KILL'
    elif roi < -20:
        return 'PAUSE'
    elif roi < 40:
        return 'CONTINUE'
    else:
        return 'SCALE'

latest_detailed['recommendation'] = latest_detailed['roi_percent'].apply(get_recommendation)

# Merge with uncertainty if available
if 'roi_uncertainty_analysis.csv' in ["D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/" + f for f in ['roi_uncertainty_analysis.csv']]:
    scorecard = latest_detailed.merge(
        df_uncertainty,
        on='initiative_id',
        how='left',
        suffixes=('', '_uncertainty')
    )
else:
    scorecard = latest_detailed.copy()
    scorecard['probability_profitable'] = 0
    scorecard['expected_12mo_loss'] = scorecard['net_burn'] * 12
    scorecard['roi_5th_percentile'] = scorecard['roi_percent'] - 10
    scorecard['roi_95th_percentile'] = scorecard['roi_percent'] + 10

scorecard.to_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/powerbi_initiative_scorecard.csv", index=False)
print("✅ Created: powerbi_initiative_scorecard.csv")

# ============================================
# DATASET 4: KILL SCENARIOS
# ============================================

current_cash = latest['cash_balance']
current_burn = latest['net_burn']
current_runway = latest['runway_months']

scenarios = []

# Baseline
scenarios.append({
    'scenario': 'Baseline (Current)',
    'initiatives_killed': 0,
    'monthly_burn_change': 0,
    'monthly_revenue_change': 0,
    'new_net_burn': current_burn,
    'new_runway': current_runway,
    'runway_extension': 0
})

# Kill each initiative individually
for idx, row in latest_detailed.iterrows():
    burn_saved = row['monthly_burn']
    revenue_lost = row['monthly_revenue']
    new_burn = current_burn - burn_saved + revenue_lost
    
    if new_burn > 0:
        new_runway = current_cash / new_burn
    else:
        new_runway = 999
    
    scenarios.append({
        'scenario': f"Kill {row['initiative_name']}",
        'initiative': row['initiative_name'],
        'initiatives_killed': 1,
        'monthly_burn_change': -burn_saved,
        'monthly_revenue_change': -revenue_lost,
        'new_net_burn': new_burn,
        'new_runway': new_runway,
        'runway_extension': new_runway - current_runway
    })

# Kill all negative ROI
kill_all_negative = latest_detailed[latest_detailed['roi_percent'] < 0]
total_burn_saved = kill_all_negative['monthly_burn'].sum()
total_revenue_lost = kill_all_negative['monthly_revenue'].sum()
new_burn_all = current_burn - total_burn_saved + total_revenue_lost

if new_burn_all > 0:
    new_runway_all = current_cash / new_burn_all
else:
    new_runway_all = 999

scenarios.append({
    'scenario': 'Kill All Negative ROI',
    'initiative': 'Multiple',
    'initiatives_killed': len(kill_all_negative),
    'monthly_burn_change': -total_burn_saved,
    'monthly_revenue_change': -total_revenue_lost,
    'new_net_burn': new_burn_all,
    'new_runway': new_runway_all,
    'runway_extension': new_runway_all - current_runway
})

df_scenarios = pd.DataFrame(scenarios)
df_scenarios.to_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/powerbi_kill_scenarios.csv", index=False)
print("✅ Created: powerbi_kill_scenarios.csv")

# ============================================
# DATASET 5: RUNWAY PROJECTION
# ============================================

# Calculate kill savings
kill_savings = total_burn_saved - total_revenue_lost
scale_profit = 56000  # Estimated from scaling Core SaaS

projections = []

for month in range(0, 25):
    # Current trajectory
    cash_current = current_cash - (current_burn * month)
    
    # Kill negative
    new_burn_kill = current_burn - kill_savings
    if new_burn_kill > 0:
        cash_kill = current_cash - (new_burn_kill * month)
    else:
        cash_kill = current_cash + (abs(new_burn_kill) * month)
    
    # Scale core
    cash_scale = current_cash - ((current_burn - scale_profit) * month)
    
    # Optimal
    optimal_burn = current_burn - kill_savings - scale_profit
    if optimal_burn > 0:
        cash_optimal = current_cash - (optimal_burn * month)
    else:
        cash_optimal = current_cash + (abs(optimal_burn) * month)
    
    projections.append({
        'months_from_now': month,
        'cash_baseline': max(cash_current, 0),
        'cash_kill_negative': cash_kill,
        'cash_scale_core': max(cash_scale, 0),
        'cash_optimal': cash_optimal
    })

df_projections = pd.DataFrame(projections)
df_projections.to_csv("D:/Projects/End-to-end projects/7. Runway Kill-Switch Analytics/Data/powerbi_runway_projection.csv", index=False)
print("✅ Created: powerbi_runway_projection.csv")

# ============================================
# SUMMARY
# ============================================

print("\n" + "="*70)
print("✅ ALL POWER BI DATASETS CREATED!")
print("="*70)

print("\nNew files created:")
print("  1. powerbi_master_metrics.csv")
print("  2. powerbi_runway_trend.csv")
print("  3. powerbi_initiative_scorecard.csv")
print("  4. powerbi_kill_scenarios.csv")
print("  5. powerbi_runway_projection.csv")

print("\n📊 Key Numbers for Dashboard:")
print(f"Current Runway: {current_runway:.1f} months")
print(f"Cash Balance: ${current_cash:,.0f}")
print(f"Monthly Burn: ${current_burn:,.0f}")
print(f"Initiatives to Kill: {len(kill_all_negative)}")
print(f"Potential Runway Extension: {new_runway_all - current_runway:.1f} months")
print(f"Monthly Savings if Kill All: ${kill_savings:,.0f}")

print("\n🎯 Now load these 5 new files into Power BI!")

GENERATING MISSING POWER BI DATASETS
✅ Loaded existing files
✅ Created: powerbi_master_metrics.csv
✅ Created: powerbi_runway_trend.csv
✅ Created: powerbi_initiative_scorecard.csv
✅ Created: powerbi_kill_scenarios.csv
✅ Created: powerbi_runway_projection.csv

✅ ALL POWER BI DATASETS CREATED!

New files created:
  1. powerbi_master_metrics.csv
  2. powerbi_runway_trend.csv
  3. powerbi_initiative_scorecard.csv
  4. powerbi_kill_scenarios.csv
  5. powerbi_runway_projection.csv

📊 Key Numbers for Dashboard:
Current Runway: 21.5 months
Cash Balance: $1,819,510
Monthly Burn: $84,736
Initiatives to Kill: 5
Potential Runway Extension: 977.5 months
Monthly Savings if Kill All: $131,460

🎯 Now load these 5 new files into Power BI!
